# a. 使用大模型 API 对视频进行快速摘要（音频处理）- 完整版
> [HW9: Quick Summary of Lecture Video (演講影片快速摘要)](https://colab.research.google.com/drive/1Ysr25kz6lP7gR8DNTkJMAqOuMp2bhXes?usp=sharing#scrollTo=UULEr1GpDAl6)中文镜像版 | [PDF](https://github.com/Hoper-J/AI-Guide-and-Demos-zh_CN/blob/master/GenAI_PDF/HW9.pdf)
>
> 这里的变量名与HW9基本保持一致。

**目标：** 学习如何使用现成的 API 快速构建语音识别相关的应用。

注意，这里没有视觉上的识别。

如果你遇到了 443 或者代理端口错误，请查看[《a. 使用 HFD 加快 Hugging Face 模型和数据集的下载》](https://github.com/Hoper-J/AI-Guide-and-Demos-zh_CN/blob/master/Guide/a.%20使用%20HFD%20加快%20Hugging%20Face%20模型和数据集的下载.md#可能存在的问题443-和-git-clone-failed)。

你也可以尝试运行 [b. 精简版]()，适合一键执行查看效果，其中只保留了核心代码。

## 第1部分 - 准备

### 提供的作业演讲视频

(1) 为了方便处理，已将其转换为 MP3 文件。

(2) 如果你想查看原始视频，请点击以下链接：

- 李琳山教授 信号与人生 (2023)

  - 视频：[Bilibili](https://www.bilibili.com/video/BV14P411B7Le) | [YouTube](https://www.youtube.com/watch?v=MxoQV4M0jY8)


(3) 由于原始演讲视频较长，选取了从 1:43:24 到 2:00:49 的片段用于此次演示。

### 安装必要的库

这里需要一点时间。

注意，Colab 的版本在本地运行可能存在一些问题，故有以下修改：
1. 官方版本下载的OpenCC是1.1.7，这可能会遇到报错：
>ImportError: /lib/x86_64-linux-gnu/libc.so.6: version `GLIBC_2.32' not found

这里将版本降到 1.1.6 避免这个问题，解决方法来源: [OpenCC Issues#832](https://github.com/BYVoid/OpenCC/issues/832)

2. 本地运行还可能遇到：
> ImportError: To support decoding audio files, please install 'librosa' and 'soundfile'.

对应增加安装指令即可。


In [ ]:
!pip install srt==3.5.3
!pip install datasets==2.20.0
!pip install DateTime==5.5
!pip install OpenCC==1.1.6
!pip install opencv-contrib-python==4.8.0.76
!pip install opencv-python==4.8.0.76
!pip install opencv-python-headless==4.10.0.84
!pip install openpyxl==3.1.4
!pip install openai==1.35.3
!pip install git+https://github.com/openai/whisper.git@ba3f3cd54b0e5b8ce1ab3de13e32122d0d5f98ab
!pip install numpy==1.25.2
!pip install soundfile==0.12.1
!pip install ipywidgets==8.0.0  # 不兼容的话可以降低版本
!pip install librosa==librosa-0.10.2.post1

### 导入

In [1]:
# 标准库
import os
import time
import re
import pathlib
import textwrap
import datetime

# 第三方库
import numpy as np
import srt
import soundfile as sf
from opencc import OpenCC
from tqdm import tqdm
import ipywidgets as widgets
from IPython.display import display, Markdown

# 项目相关库（如语音识别和API调用等）
import whisper
from datasets import load_dataset
from openai import OpenAI

### 下载数据

已经上传至 data 文件夹中，不需要再下载。

In [2]:
# 加载数据集
# dataset_name = "kuanhuggingface/NTU-GenAI-2024-HW9"
# dataset = load_dataset(dataset_name)


# 加载本地 Parquet 格式的数据集
dataset = load_dataset('parquet', data_files={'test': './data/13/test-00000-of-00001.parquet'})

In [3]:
# 准备音频
input_audio = dataset["test"]["audio"][0]
input_audio_name = input_audio["path"]
input_audio_array = input_audio["array"].astype(np.float32)
sampling_rate = input_audio["sampling_rate"]

print(f"现在我们将转录音频: 李琳山教授 信号与人生 (2023) ({input_audio_name})。")

现在我们将转录音频: 李琳山教授 信号与人生 (2023) (ntu-gen-ai-2024-hw9-16k.mp3)。


## 第2部分 - 自动语音识别 (ASR)

函数 `speech_recognition()` 旨在将音频转换为字幕。

In [4]:
def speech_recognition(model_name, input_audio, output_subtitle_path, decode_options, cache_dir="./"):
    '''
        (1) 目标:
            - 本函数旨在将音频转换为字幕。

        (2) 参数:

            - model_name (str):
                模型名称。模型有五种尺寸，包括 tiny, base, small, medium, large-v3。
                例如，你可以使用 'tiny', 'base', 'small', 'medium', 'large-v3' 来指定模型名称。
                详细信息请见 'https://github.com/openai/whisper'。

            - input_audio (Union[str, np.ndarray, torch.Tensor]):
                音频文件路径或音频波形。
                - 例如，如果输入音频路径为 'input.wav'，你可以使用 'input.wav' 来指定音频路径。
                - 如果输入音频数组为 'audio_array'，你可以使用 'audio_array' 来指定音频数组。

            - output_subtitle_path (str):
                输出字幕文件路径。
                例如，如果你想将字幕文件保存为 'output.srt'，可以使用 'output.srt' 来指定路径。

            - decode_options (dict):
                音频文件解码选项，包括 'initial_prompt', 'prompt', 'prefix', 'temperature'。
                - initial_prompt (str):
                    提供给模型的初始文本prompt。这可以用于提供上下文，以提高识别准确性。
                    默认值: None。

                详细信息请见 "https://github.com/openai/whisper/blob/main/whisper/decoding.py" 和 "https://github.com/openai/whisper/blob/main/whisper/transcribe.py"。

                - temperature (float):
                    采样的温度。值越高，随机性越强。
                    默认值: 0.0

            - cache_dir (str):
                用于保存模型的缓存目录路径。
                例如，你可以使用 'cache' 来指定缓存目录。
                默认值: './'

        (3) 示例:

            - 如果你想使用 'base' 模型将 'input.wav' 转换为 'output.srt'，并将缓存文件保存在 'cache' 目录中，
              你可以按如下方式调用此函数：

                speech_recognition(model_name='base', input_audio_path='input.wav', output_subtitle_path='output.srt', cache_dir='cache')
    '''

    # 记录开始时间
    start_time = time.time()

    print(f"=============== 正在加载 Whisper-{model_name} ===============")

    # 加载模型
    model = whisper.load_model(name=model_name, download_root=cache_dir)

    print(f"开始使用 Whisper-{model_name} 转录音频。")

    # 转录音频
    transcription = model.transcribe(audio=input_audio, language=decode_options["language"], verbose=False,
                                     initial_prompt=decode_options["initial_prompt"], temperature=decode_options["temperature"])

    # 记录结束时间
    end_time = time.time()

    print(f"语音识别过程耗时 {end_time - start_time} 秒。")

    subtitles = []
    # 将转录内容转换为字幕并遍历所有片段
    for i, segment in tqdm(enumerate(transcription["segments"])):

        # 将开始时间转换为字幕格式
        start_time = datetime.timedelta(seconds=segment["start"])

        # 将结束时间转换为字幕格式
        end_time = datetime.timedelta(seconds=segment["end"])

        # 获取字幕文本
        text = segment["text"]

        # 将字幕添加到字幕列表中
        subtitles.append(srt.Subtitle(index=i, start=start_time, end=end_time, content=text))

    # 将字幕列表转换为字幕内容
    srt_content = srt.compose(subtitles)

    print(f"\n=============== 正在将字幕保存到 {output_subtitle_path} ===============")

    # 将字幕内容保存到文件中
    with open(output_subtitle_path, "w", encoding="utf-8") as file:
        file.write(srt_content)

    print(f"\n=============== 字幕保存完成 ===============")
    

直接运行代码，交互式修改所需的参数及输入文件路径，不用关心这里的代码细节，除非你对交互感兴趣。

In [5]:
# ===== 在此代码块中，你可以修改所需的参数及输入文件路径 =====

# 模型名称部分
model_name_markdown = Markdown("**model_name**<br>请选择要使用的模型名称：")
model_name_widget = widgets.Dropdown(
    options=['tiny', 'base', 'small', 'medium', 'large-v3'],
    value='medium',
    description='model_name:',
)

# 输出文件的后缀部分
suffix_markdown = Markdown("**suffix**<br>设置输出文件名的后缀：")
suffix_widget = widgets.Text(
    value='信号与人生',
    description='suffix:',
)

# 输出字幕文件和原始文本文件路径部分
output_subtitle_path_widget = widgets.Text(
    value=f"./output-信号与人生.srt",
    description='字幕路径:',
    disabled=True  # 禁用手动修改
)

output_raw_text_path_widget = widgets.Text(
    value=f"./output-信号与人生.txt",
    description='原始文本路径:',
    disabled=True  # 禁用手动修改
)

# 模型和数据集缓存目录部分
cache_dir_markdown = Markdown("**cache_dir**<br>设置模型和数据集缓存的目录路径：")
cache_dir_widget = widgets.Text(
    value='./',
    description='cache_dir:',
)

# 演讲视频的语言部分
language_markdown = Markdown("**language**<br>设置演讲视频的语言：")
language_widget = widgets.Text(
    value='zh',
    description='language:',
)

# 用于提供初始文本prompt（可选）
initial_prompt_markdown = Markdown("**initial_prompt**<br>用于提供初始文本prompt（可选）：")
initial_prompt_widget = widgets.Text(
    value='请用中文',
    description='initial_prompt:',
)

# 采样温度部分
temperature_markdown = Markdown("**temperature**<br>值越高，随机性越强：")
temperature_widget = widgets.FloatSlider(
    value=0,
    min=0,
    max=1,
    step=0.1,
    description='temperature:',
    continuous_update=False
)

# 创建输出区域来显示打印内容
output_area = widgets.Output()

# 获取用户输入值并更新路径
def on_button_click(b):
    global model_name, suffix, output_subtitle_path, output_raw_text_path, cache_dir, language, initial_prompt, temperature
    with output_area:  # 使用 Output 小部件捕获输出
        output_area.clear_output()  # 清除之前的输出
        model_name = model_name_widget.value
        suffix = suffix_widget.value
        output_subtitle_path = f"./output-{suffix}.srt"  # 更新全局变量
        output_raw_text_path = f"./output-{suffix}.txt"  # 更新全局变量
        cache_dir = cache_dir_widget.value
        language = language_widget.value
        initial_prompt = initial_prompt_widget.value
        temperature = temperature_widget.value
        
        # 更新字幕文件路径和原始文本路径
        output_subtitle_path_widget.value = output_subtitle_path
        output_raw_text_path_widget.value = output_raw_text_path
        
        # 打印配置
        print(f"模型名称: {model_name}")
        print(f"输出文件后缀: {suffix}")
        print(f"字幕文件路径: {output_subtitle_path}")
        print(f"原始文本路径: {output_raw_text_path}")
        print(f"缓存目录: {cache_dir}")
        print(f"语言: {language}")
        print(f"初始prompt: {initial_prompt}")
        print(f"采样温度: {temperature}")

# 创建提交按钮
submit_button = widgets.Button(description="提交配置")
submit_button.on_click(on_button_click)

# 显示带有说明的所有小部件
display(model_name_markdown, model_name_widget)
display(suffix_markdown, suffix_widget)
display(output_subtitle_path_widget, output_raw_text_path_widget)
display(cache_dir_markdown, cache_dir_widget)
display(language_markdown, language_widget)
display(initial_prompt_markdown, initial_prompt_widget)
display(temperature_markdown, temperature_widget)

# 显示提交按钮
display(submit_button)

# 显示输出区域
display(output_area)

**model_name**<br>请选择要使用的模型名称：

Dropdown(description='model_name:', index=3, options=('tiny', 'base', 'small', 'medium', 'large-v3'), value='m…

**suffix**<br>设置输出文件名的后缀：

Text(value='信号与人生', description='suffix:')

Text(value='./output-信号与人生.srt', description='字幕路径:', disabled=True)

...

Text(value='请用中文', description='initial_prompt:')

**temperature**<br>值越高，随机性越强：

FloatSlider(value=0.0, continuous_update=False, description='temperature:', max=1.0)

Button(description='提交配置', style=ButtonStyle())

Output()

构建 DecodingOptions

In [6]:
decode_options = {
    "language": language,
    "initial_prompt": initial_prompt,
    "temperature": temperature
}

**打印完整配置**

In [7]:
message = "转录 李琳山教授 信号与人生 (2023)"
print(f"设置: (1) 模型: whisper-{model_name} (2) 语言: {language} (3) 初始prompt: {initial_prompt} (4) 温度: {temperature}")
print(message)

设置: (1) 模型: whisper-medium (2) 语言: zh (3) 初始prompt: 请用中文 (4) 温度: 0.0
转录 李琳山教授 信号与人生 (2023)


预计需要花费 4 分钟时间（包括下载和识别，识别大概1分钟）（由模型大小，网速和你的显卡决定）

下载+识别：

![结果](../Guide/assets/image-20240924005355363.png)

In [8]:
# 运行 ASR
speech_recognition(model_name=model_name, input_audio=input_audio_array, output_subtitle_path=output_subtitle_path, decode_options=decode_options, cache_dir=cache_dir)

=============== 正在加载 Whisper-medium ===============
开始使用 Whisper-medium 转录音频。


100%|███████████████████████████████████████████████████████████| 104500/104500 [01:10<00:00, 1487.98frames/s]


语音识别过程耗时 75.505859375 秒。


385it [00:00, 801393.07it/s]


=============== 正在将字幕保存到 ./output-信号与人生.srt ===============

=============== 字幕保存完成 ===============


**检查结果**

In [9]:
''' 打开 SRT 文件并读取其内容。
SRT 文件格式为：

[索引]
[开始时间] (小时:分钟:秒) --> [结束时间] (小时:分钟:秒)
[转录内容]
'''

with open(output_subtitle_path, 'r', encoding='utf-8') as file:
    content = file.read()

print(content)

1
00:00:00,000 --> 00:00:04,000
每次说 学问是做出来的

2
00:00:06,000 --> 00:00:08,000
什么意思

3
00:00:08,000 --> 00:00:12,000
要做 才会获得学问

4
00:00:13,000 --> 00:00:16,000
你如果每天光是坐在那里听

5
00:00:17,000 --> 00:00:20,000
学问很可能是左耳近右耳出的

6
00:00:21,000 --> 00:00:23,000
你光是坐在那儿读

7
00:00:23,000 --> 00:00:26,000
学问可能从眼睛进入脑海之后就忘掉了

8
00:00:26,000 --> 00:00:29,000
如何能够学问在脑海里面

9
00:00:31,000 --> 00:00:33,000
真的变成你自己学问

10
00:00:33,000 --> 00:00:35,000
就是要做

11
00:00:36,000 --> 00:00:39,000
可能有很多同学有这个经验

12
00:00:39,000 --> 00:00:41,000
你如果去修某一门课

13
00:00:41,000 --> 00:00:44,000
或者做某一个实验

14
00:00:44,000 --> 00:00:47,000
在期末就是要教一个final project

15
00:00:48,000 --> 00:00:50,000
那个final project就是要你把

16
00:00:51,000 --> 00:00:53,000
学到的很多东西

17
00:00:53,000 --> 00:00:56,000
最后整合在你的final project里面

18
00:00:56,000 --> 00:00:58,000
最后做出来的时候

19
00:00:58,000 --> 00:01:00,000
就是把它们都整合了

20
00:01:00,000 --> 00:01:02,000
当你学期结束

21
00:01:02,000 --> 00:01:04,000
真的把final project做完的时候

22
00:01:04,000 --> 00:01:05,000

In [10]:
def extract_and_save_text(srt_filename, output_filename, convert_to_tradition_chinese=False):
    '''
    (1) 目标:
        - 该函数提取SRT文件中的文本并将其保存到新的文本文件中。
        - 如果 convert_to_tradition_chinese 为 True，则将简体中文转换为繁体中文。

    (2) 参数:

        - srt_filename: SRT文件的路径。

        - output_filename: 输出文本文件的名称。

        - convert_to_tradition_chinese: 布尔值，指示是否将简体中文转换为繁体中文。默认值为 False。

    (3) 示例:
        - 如果你的SRT文件名为 'subtitle.srt'，并且希望将提取的文本保存为名为 'output.txt' 的文件，不进行简繁转换，可以按如下方式使用该函数:
            extract_and_save_text('subtitle.srt', 'output.txt')

        - 如果需要将简体中文转换为繁体中文，可以设置 convert_to_tradition_chinese=True:
            extract_and_save_text('subtitle.srt', 'output.txt', convert_to_tradition_chinese=True)
    '''

    # 打开SRT文件并读取其内容
    with open(srt_filename, 'r', encoding='utf-8') as file:
        content = file.read()

    # 使用正则表达式去除时间码
    pure_text = re.sub(r'\d+\n\d{2}:\d{2}:\d{2},\d{3} --> \d{2}:\d{2}:\d{2},\d{3}\n', '', content)

    # 去除空行
    pure_text = re.sub(r'\n\n+', '\n', pure_text)

    # 如果 convert_to_tradition_chinese 为 True，则将简体中文转换为繁体中文
    if convert_to_tradition_chinese:
        cc = OpenCC('s2t')
        pure_text = cc.convert(pure_text)

    # 将提取的文本写入新文件
    with open(output_filename, 'w', encoding='utf-8') as output_file:
        output_file.write(pure_text)

    print(f'提取的文本已保存到 {output_filename}。\n\n')

    return pure_text

## 第3部分 - 处理自动语音识别的结果



In [11]:
def chunk_text(text, max_length):
    """
    (1) 目标:
        - 此函数用于将长字符串拆分为指定长度的较小字符串。

    (2) 参数:
        - text: str, 要拆分的长字符串。
        - max_length: int, 每个较小字符串的最大长度。

    (3) 返回:
        - split_text: list, 拆分后的字符串列表。

    (3) 示例:
        - 如果你想将一个名为 "long_string" 的字符串拆分为长度为 100 的较小字符串，可以按如下方式使用该函数:
            chunk_text(long_string, 100)
    """

    return textwrap.wrap(text, max_length)

这里可以自定义修改文本块参数，修改后执行下一段代码查看结果。

如果你更偏好繁体，修改 convert_to_tradition_chinese = True。

In [12]:
# 文本块的长度
chunk_length = 512

# 决定是否转为繁体
convert_to_tradition_chinese = False

In [13]:
# 从SRT文件中提取文本并保存为新文本文件
pure_text = extract_and_save_text(srt_filename=output_subtitle_path, output_filename=output_raw_text_path, convert_to_tradition_chinese=convert_to_tradition_chinese)

# 将长文本拆分为指定长度的较小块
chunks = chunk_text(text=pure_text, max_length=512)

# 你可以查看每段的字数和内容
print("查看将长文本拆分为多个短文本的结果。\n")
for index, chunk in enumerate(chunks):
    if index == 0:
        print(f"\n========== 拆分后的第 {index + 1} 段（{len(chunk)} 字） ==========\n\n")
        for text in textwrap.wrap(chunk, 80):
            print(f"{text}\n")
    elif index == 1:
        print(f"\n========== 拆分后的第 {index + 1} 段（{len(chunk)} 字） ==========\n\n")
        for text in textwrap.wrap(chunk, 80):
            print(f"{text}\n")
    elif index == 2:
        print(f"\n========== 拆分后的第 {index + 1} 段（{len(chunk)} 字） ==========\n\n")
        for text in textwrap.wrap(chunk, 80):
            print(f"{text}\n")
    else:
        print(f"\n========== 拆分后的第 {index + 1} 段（{len(chunk)} 字） ==========\n\n")
        for text in textwrap.wrap(chunk, 80):
            print(f"{text}\n")

提取的文本已保存到 ./output-信号与人生.txt。


查看将长文本拆分为多个短文本的结果。


========== 拆分后的第 1 段（506 字） ==========


每次说 学问是做出来的 什么意思 要做 才会获得学问 你如果每天光是坐在那里听 学问很可能是左耳近右耳出的 你光是坐在那儿读 学问可能从眼睛进入脑海之后就忘掉了

如何能够学问在脑海里面 真的变成你自己学问 就是要做 可能有很多同学有这个经验 你如果去修某一门课 或者做某一个实验 在期末就是要教一个final

project 那个final project就是要你把 学到的很多东西 最后整合在你的final project里面 最后做出来的时候 就是把它们都整合了

当你学期结束 真的把final project做完的时候 你会忽然发现 我真的学到很多东西 那就是做出来的学问 也许可以举另外一个例子 就是你如果学了某一些

很复杂的演算法或者什么 好像觉得那些不见得在你的脑海里 可是后来老师出了个习题 那个习题叫你写一个很大的程式 要把所有东西都包进去 当你把这个程式写完的时候

你会发现 你忽然把演算法 所有东西都弄通了 那就是学问是做出来的 所以我们永远要记得 尽量多动手多做 在动手跟做的过程之中 学问才可以变成是自己的

同样的情形就是说 很多时候这样动手或者做的 表现或者成绩 没有一个成绩单上的数字


========== 拆分后的第 2 段（500 字） ==========


使得很多人觉得那不重要 很多人甚至觉得 这门课要做final project 我就不修了太累了 或者说那门课需要 怎么样太累我就不要做了 而不知道

其实那个才是让你做的机会 然后可以学到最多 也就是说虽然很可能 那么辛苦的做很多事 没有让你获得什么具体成绩 对你的overfitting可能没有帮助

可是对你的全面学习 是很有帮助是该学的 不要漏掉这些事 这是我所说的 这个课业内可以做的这些事 刚才我们讲到思考的时候 我觉得我漏掉一点 你如果修我的信号课

你可能会发现 我上课没讲到一个数学试字的时候 我通常都不推它的 我是在解释那个数学试字在说什么话 同样的没讲到一个什么事情的时候 我通常就在解释他在说什么话

也就是说我在讲的就是 我读到特本那里的时候 我心里怎么想的 也就是我在告诉同

## 第4部分 - 摘要

这里提供的方法同 ChatGPT，因为一样使用 OpenAI 库进行。

你可以通过[《00. 阿里大模型API获取步骤》](https://github.com/Hoper-J/AI-Guide-and-Demos-zh_CN/blob/master/Guide/00.%20阿里大模型API获取步骤.md)获取 API 密钥。

其他模型可以参阅[模型广场 -- 阿里云百炼](https://bailian.console.aliyun.com/?spm=5176.29619931.J__Z58Z6CX7MY__Ll8p1ZOR.1.4d1d59fcWwSqvr#/model-market)，点击对应模型的`查看详情`。

![查看详情](../Guide/assets/image-20240924091151684.png)

在界面左上角可以看到对应的英文名称。

![对应模型名](../Guide/assets/image-20240924091414350.png)

复制它。

现在你可以随意更换为你想要的模型，不过你可能要先申请使用（通过大概要几个小时，会有短信提示）。

In [14]:
def summarization(client, summarization_prompt, model_name="qwen-turbo", temperature=0.0, top_p=1.0, max_tokens=512):
    """
    (1) 目标:
        - 使用 OpenAI Chat API 对给定文本进行摘要。

    (2) 参数:
        - client: OpenAI Chat API客户端。
        - summarization_prompt: 包含需要摘要的文本的摘要prompt。
        - model_name: 模型名称，默认是 "qwen-turbo"。
        - temperature: 控制响应的随机性。较低的值使响应更具确定性，默认是 0.0。
        - top_p: 通过核采样控制多样性。较高的值会导致更多样化的响应，默认是 1.0。
        - max_tokens: 完成任务时生成的最大标记数，默认是 512。

    (3) 返回:
        - 摘要文本。

    (4) 示例:
        - 如果文本是 "ABC"，摘要prompt是 "DEF"，model_name 是 "qwen-turbo"，temperature 是 0.0，top_p 是 1.0，max_tokens 是 512，则可以按如下方式调用此函数:

              summarization(client=client, text="ABC", summarization_prompt="DEF", model_name="qwen-turbo", temperature=0.0, top_p=1.0, max_tokens=512)
    """

    # 用户prompt是 summarization_prompt 和文本的结合
    user_prompt = summarization_prompt

    while True:

        try:
            # 使用 OpenAI Chat API 对文本进行摘要
            chat_completion = client.chat.completions.create(
                messages=[
                    {
                        "role": "user",
                        "content": user_prompt,
                    }
                ],
                    model=model_name,
                    temperature=temperature,
                    top_p=top_p,
                    max_tokens=max_tokens
            )

            break

        except:
            print("API调用失败，等待1秒后重试。")
            time.sleep(1)

    return chat_completion.choices[0].message.content

直接运行代码，交互式修改参数和 API，不用关心这里的代码细节。

In [15]:
# ===== 在此代码块中，你可以修改所需的参数并设置OpenAI API密钥 =====

# 你的OpenAI API密钥部分
api_key_markdown = Markdown("**openai_api_key**<br>请输入你的 OpenAI API 密钥：")
openai_api_key_widget = widgets.Password(
    value='',
    description='openai_api_key:',
    placeholder='请输入你的OpenAI API密钥',
    style={'description_width': 'initial'}
)

# 模型名称部分
model_name_markdown = Markdown("**model_name**<br>请选择模型名称，默认是 'qwen-turbo'：")
model_name_widget = widgets.Text(
    value='qwen-turbo',
    description='model_name:',
    style={'description_width': 'initial'}
)

# 控制响应的随机性部分
temperature_markdown = Markdown("**temperature**<br>通过温度值来控制响应的随机性，较低的值使响应更具确定性：")
temperature_widget = widgets.FloatSlider(
    value=0,
    min=0,
    max=1,
    step=0.1,
    description='temperature (随机性):',
    continuous_update=False,
    style={'description_width': 'initial'}
)

# 控制多样性 (Top-P) 部分
top_p_markdown = Markdown("**控制多样性 (Top-P)**<br>通过 Top-P 核采样控制多样性，较高的值会导致更多样化的响应：")
top_p_widget = widgets.FloatSlider(
    value=0,
    min=0,
    max=1,
    step=0.1,
    description='Top-P (多样性):',
    continuous_update=False,
    style={'description_width': 'initial'}
)

# 创建输出区域来显示打印内容
output_area = widgets.Output()

# 获取用户输入值并显示
def on_button_click(b):
    with output_area:
        global openai_api_key, model_name, temperature, top_p
        output_area.clear_output()  # 清除之前的输出
        openai_api_key = openai_api_key_widget.value
        model_name = model_name_widget.value
        temperature = temperature_widget.value
        top_p = top_p_widget.value
        
        # 打印配置
        print(f"OpenAI API密钥: {'*' * len(openai_api_key)}")  # 为了安全性，只显示密钥长度
        print(f"模型名称: {model_name}")
        print(f"温度 (随机性): {temperature}")
        print(f"Top-P (多样性): {top_p}")

# 创建提交按钮
submit_button = widgets.Button(description="提交配置")
submit_button.on_click(on_button_click)

# 显示带有说明的所有小部件
display(api_key_markdown, openai_api_key_widget)
display(model_name_markdown, model_name_widget)
display(temperature_markdown, temperature_widget)
display(top_p_markdown, top_p_widget)

# 显示提交按钮
display(submit_button)

# 显示输出区域
display(output_area)

**openai_api_key**<br>请输入你的 OpenAI API 密钥：

Password(description='openai_api_key:', placeholder='请输入你的OpenAI API密钥', style=TextStyle(description_width='in…

**model_name**<br>请选择模型名称，默认是 'qwen-turbo'：

Text(value='qwen-turbo', description='model_name:', style=TextStyle(description_width='initial'))

**temperature**<br>通过温度值来控制响应的随机性，较低的值使响应更具确定性：

FloatSlider(value=0.0, continuous_update=False, description='temperature (随机性):', max=1.0, style=SliderStyle(d…

**控制多样性 (Top-P)**<br>通过 Top-P 核采样控制多样性，较高的值会导致更多样化的响应：

FloatSlider(value=0.0, continuous_update=False, description='Top-P (多样性):', max=1.0, style=SliderStyle(descrip…

Button(description='提交配置', style=ButtonStyle())

Output()

In [16]:
# 构建 OpenAI 客户端
client = OpenAI(
    api_key=openai_api_key,
    base_url="https://dashscope.aliyuncs.com/compatible-mode/v1", # 这里使用的是阿里云的大模型，如果需要使用其他平台，请参考对应的开发文档后对应修改。如果使用 GPT 的 API，删除这行就可以直接运行。
)

#### 这里演示两种摘要方式
参考：[Text Summarization of Large Documents using LangChain 🦜🔗](https://github.com/GoogleCloudPlatform/generative-ai/blob/main/language/use-cases/document-summarization/summarization_large_documents_langchain.ipynb)

分别对应于 `MapReduce` 和 `Refine`，我会在之后具体讲解它们的区别，你可以通过接下来的代码来感受二者直观的区别。

不要修改 `<text>`，这是一个占位符，如果你不记得占位符的概念，回看[《03. 进阶指南：自定义 Prompt 提升大模型解题能力》](https://github.com/Hoper-J/AI-Guide-and-Demos-zh_CN/blob/master/Guide/03.%20进阶指南：自定义%20Prompt%20提升大模型解题能力.md#设计prompt解决数学问题)。

##### 多段摘要方法（Multi-Stage Summarization）- MapReduce

![image.png](../Guide/assets/image-20240924092040340.png)

1. 将长文本分成多个较小的部分，并分别获取每个小段落的摘要

In [17]:
# ===== 多段摘要：段落设置 =====

# 你可以修改摘要prompt和生成的最大标记数，但不要修改 <text> 部分
instructions = Markdown("**多段摘要：段落设置**")

# 设置最大生成标记数
max_tokens_markdown = Markdown("**max_tokens**<br>设置生成的最大标记数：")
max_tokens_widget = widgets.IntText(
    value=350,
    description='max_tokens:',
    style={'description_width': 'initial'}
)

# 摘要prompt模板
summarization_prompt_markdown = Markdown("**summarization_prompt_template**<br>你可以修改摘要prompt，但不要修改 `<text>` 部分：")
summarization_prompt_template_widget = widgets.Textarea(
    value="用 300 个字内写出这段文字的摘要，其中包括要点和所有重要细节：<text>",
    description='summarization_prompt_template:',
    layout=widgets.Layout(width='500px', height='100px'),
    style={'description_width': 'initial'}
)

# 创建输出区域来显示打印内容
output_area = widgets.Output()

# 获取用户输入值并显示
def on_button_click(b):
    global max_tokens, summarization_prompt_template
    with output_area:
        output_area.clear_output()  # 清除之前的输出
        max_tokens = max_tokens_widget.value
        summarization_prompt_template = summarization_prompt_template_widget.value
        
        # 打印配置
        print(f"最大标记数: {max_tokens}")
        print(f"摘要prompt模板: {summarization_prompt_template}")

# 创建提交按钮
submit_button = widgets.Button(description="提交配置")
submit_button.on_click(on_button_click)

# 显示带有说明的所有小部件
display(instructions)
display(max_tokens_markdown, max_tokens_widget)
display(summarization_prompt_markdown, summarization_prompt_template_widget)

# 显示提交按钮
display(submit_button)

# 显示输出区域
display(output_area)

**多段摘要：段落设置**

**max_tokens**<br>设置生成的最大标记数：

IntText(value=350, description='max_tokens:', style=DescriptionStyle(description_width='initial'))

**summarization_prompt_template**<br>你可以修改摘要prompt，但不要修改 `<text>` 部分：

Textarea(value='用 300 个字内写出这段文字的摘要，其中包括要点和所有重要细节：<text>', description='summarization_prompt_template:', layout…

Button(description='提交配置', style=ButtonStyle())

Output()

In [18]:
paragraph_summarizations = []

# 首先，我们分别对每个拆分的部分进行摘要。
for index, chunk in enumerate(chunks):

    # 记录开始时间
    start = time.time()

    # 构建摘要prompt
    summarization_prompt = summarization_prompt_template.replace("<text>", chunk)

    # 分别对每个拆分的部分进行摘要
    response = summarization(client=client, summarization_prompt=summarization_prompt, model_name=model_name, temperature=temperature, top_p=top_p, max_tokens=max_tokens)

    # 计算执行时间，并保留两位小数
    cost_time = round(time.time() - start, 2)

    # 打印摘要及其长度
    print(f"----------------------------第 {index + 1} 段的摘要----------------------------\n")
    for text in textwrap.wrap(response, 80):
        print(f"{text}\n")
    print(f"第 {index + 1} 段摘要的长度: {len(response)}")
    print(f"生成第 {index + 1} 段摘要的时间: {cost_time} 秒。\n")

    # 记录结果
    paragraph_summarizations.append(response)

----------------------------第 1 段的摘要----------------------------

这段文字强调了"学问是做出来的"这一观点。核心思想是通过实践操作获取知识比单纯听讲或阅读更有效。作者指出，只听不练会导致知识遗忘，只读不练则难以真正内化为个人学

问。具体来说，通过完成课程作业、实验项目或大型编程任务等实际操作，可以将所学知识整合并深化理解。这种实践性学习方式有助于学生在学期末完成最终项目时，发现自己确实

学到了很多东西，即所谓的"做出来的学问"。以学习复杂算法为例，通过完成包含所有相关元素的大规模程序设计任务，学生能够将其理论知识转化为实际技能，从而实现对算法的

全面理解和掌握。因此，文章强调了实践操作的重要性，鼓励大家尽量动手实践，因为这样的学习过程才能使知识真正成为个人的学问，而不仅仅是成绩单上的分数。

第 1 段摘要的长度: 313
生成第 1 段摘要的时间: 4.43 秒。

----------------------------第 2 段的摘要----------------------------

这段文字强调了参加困难课程的重要性及其潜在价值。很多人因为担心课程过于繁重或耗时而选择避免，但实际上，正是这些挑战性课程提供了最深入的学习机会。作者指出，尽管这

类课程可能不会立即带来具体的成绩提升，但它们对于整体学习过程至关重要，有助于避免知识过拟合，促进全面学习。作者特别提到了在自己的信号课程中，他侧重于解释数学公式

背后的含义而非直接推导，鼓励学生通过阅读和思考来理解概念，从而培养批判性思维能力。他认为，将阅读过程中遇到的每个数学公式或理论作为思考练习，是提升思维能力和形成

良好思考习惯的最佳方式。总之，这段文字强调了面对挑战性学习机会的价值，鼓励读者不应因暂时的困难而放弃，而应将其视为个人成长和学习的重要途径。

第 2 段摘要的长度: 310
生成第 2 段摘要的时间: 4.15 秒。

----------------------------第 3 段的摘要----------------------------

这段文字强调了广泛的学习概念，不仅局限于学术课程，还包括生活中的各种活动。关键要点包括：  1.

**学习的本质**：学习被定义为增长、进步并从中获得快乐的过程。  2

In [19]:
# 收集之前获得的所有摘要并打印它们。

collected_summarization = ""
for index, paragraph_summarization in enumerate(paragraph_summarizations):
    collected_summarization += f"第 {index + 1} 段的摘要: {paragraph_summarization}\n"

print(collected_summarization)

第 1 段的摘要: 这段文字强调了"学问是做出来的"这一观点。核心思想是通过实践操作获取知识比单纯听讲或阅读更有效。作者指出，只听不练会导致知识遗忘，只读不练则难以真正内化为个人学问。具体来说，通过完成课程作业、实验项目或大型编程任务等实际操作，可以将所学知识整合并深化理解。这种实践性学习方式有助于学生在学期末完成最终项目时，发现自己确实学到了很多东西，即所谓的"做出来的学问"。以学习复杂算法为例，通过完成包含所有相关元素的大规模程序设计任务，学生能够将其理论知识转化为实际技能，从而实现对算法的全面理解和掌握。因此，文章强调了实践操作的重要性，鼓励大家尽量动手实践，因为这样的学习过程才能使知识真正成为个人的学问，而不仅仅是成绩单上的分数。
第 2 段的摘要: 这段文字强调了参加困难课程的重要性及其潜在价值。很多人因为担心课程过于繁重或耗时而选择避免，但实际上，正是这些挑战性课程提供了最深入的学习机会。作者指出，尽管这类课程可能不会立即带来具体的成绩提升，但它们对于整体学习过程至关重要，有助于避免知识过拟合，促进全面学习。作者特别提到了在自己的信号课程中，他侧重于解释数学公式背后的含义而非直接推导，鼓励学生通过阅读和思考来理解概念，从而培养批判性思维能力。他认为，将阅读过程中遇到的每个数学公式或理论作为思考练习，是提升思维能力和形成良好思考习惯的最佳方式。总之，这段文字强调了面对挑战性学习机会的价值，鼓励读者不应因暂时的困难而放弃，而应将其视为个人成长和学习的重要途径。
第 3 段的摘要: 这段文字强调了广泛的学习概念，不仅局限于学术课程，还包括生活中的各种活动。关键要点包括：

1. **学习的本质**：学习被定义为增长、进步并从中获得快乐的过程。

2. **课业外学习的重要性**：指出课业外的活动同样能够带来学习与成长，不论是对个人技能、身体健康、情感智力还是社交能力的提升。

3. **具体实例**：
   - **运动**：如打球，不仅能促进健康，还能提高手脑协调性、团队合作及人际交往能力。
   - **爬山**：不仅锻炼身体，还能教会坚韧不拔的精神、自然观察力等。
   - **旅行**：有助于拓宽视野、增加知识、增进自我理解，以及体验不同文化。

4. **快乐与学习的关系**：强调学习过程中获得的快乐是衡量其价值的重要指标，鼓励将任何带来快乐的经历视

2. 在分别获取每个小段落的摘要后，处理这些摘要以生成最终的摘要。

直接运行代码，交互式修改，不用关心这里的代码细节。

In [20]:
# ===== 多段摘要：总摘要设置 =====

# 你可以修改摘要prompt和生成的最大标记数，但不要修改 <text> 部分
instructions = Markdown("**多段摘要：总摘要**")

# 设置生成的最大标记数
max_tokens_markdown = Markdown("**max_tokens**<br>我们设置生成的最大标记数，确保最终摘要不超过 550 个标记。")
max_tokens_widget = widgets.IntText(
    value=550,
    description='max_tokens:',
    style={'description_width': 'initial'}
)

# 摘要prompt模板
summarization_prompt_markdown = Markdown("**summarization_prompt**<br>你可以修改摘要prompt，但不要修改 `<text>` 部分：")
summarization_prompt_widget = widgets.Textarea(
    value="在500字以内写出以下文字的简洁摘要：<text>",
    description='summarization_prompt:',
    layout=widgets.Layout(width='500px', height='100px'),
    style={'description_width': 'initial'}
)

# 创建输出区域来显示打印内容
output_area = widgets.Output()

# 获取用户输入值并显示
def on_button_click(b):
    global max_tokens, summarization_prompt_template
    with output_area:
        output_area.clear_output()  # 清除之前的输出
        max_tokens = max_tokens_widget.value
        summarization_prompt_template = summarization_prompt_widget.value
        
        # 打印配置
        print(f"最大标记数: {max_tokens}")
        print(f"摘要prompt模板: {summarization_prompt_template}")

# 创建提交按钮
submit_button = widgets.Button(description="提交配置")
submit_button.on_click(on_button_click)

# 显示带有说明的所有小部件
display(instructions)
display(max_tokens_markdown, max_tokens_widget)
display(summarization_prompt_markdown, summarization_prompt_widget)

# 显示提交按钮
display(submit_button)

# 显示输出区域
display(output_area)

**多段摘要：总摘要**

**max_tokens**<br>我们设置生成的最大标记数，确保最终摘要不超过 550 个标记。

IntText(value=550, description='max_tokens:', style=DescriptionStyle(description_width='initial'))

**summarization_prompt**<br>你可以修改摘要prompt，但不要修改 `<text>` 部分：

Textarea(value='在500字以内写出以下文字的简洁摘要：<text>', description='summarization_prompt:', layout=Layout(height='100px',…

Button(description='提交配置', style=ButtonStyle())

Output()

实际运行时间与 API 有关。

In [21]:
# 记录开始时间
start = time.time()

# 运行最终摘要生成
summarization_prompt = summarization_prompt_template.replace("<text>", collected_summarization)
final_summarization = summarization(client=client, summarization_prompt=summarization_prompt, model_name=model_name, temperature=temperature, top_p=top_p, max_tokens=max_tokens)

# 计算执行时间并四舍五入到两位小数
cost_time = round(time.time() - start, 2)

# 打印摘要及其长度
print(f"----------------------------最终摘要----------------------------\n")
for text in textwrap.wrap(final_summarization, 80):
        print(f"{text}")
print(f"\n最终摘要的长度: {len(final_summarization)}")
print(f"生成最终摘要所需的时间: {cost_time} 秒。")

----------------------------最终摘要----------------------------

第一段摘要：文章强调实践操作在获取知识中的重要性，主张通过动手操作如完成课程作业、实验项目或编程任务，比单纯听讲或阅读更能有效吸收知识。实践性学习能加深理解，帮
助学生在学期末通过完成最终项目时验证学习成果，即“做出来的学问”。以学习复杂算法为例，大规模程序设计任务可将理论知识转化为实际技能，实现全面理解和掌握。文章鼓励
实践操作，认为这样的学习过程使知识成为个人学问，而非仅停留在分数层面。  第二段摘要：文章强调参加挑战性课程的价值，指出此类课程虽然短期内可能不易带来成绩提升，
但对整体学习过程至关重要，有助于避免知识过拟合，促进全面学习。作者以自己在信号课程中的教学方式为例，侧重解释数学公式的含义而非直接推导，鼓励学生通过阅读和思考理
解概念，培养批判性思维能力。作者认为，将阅读过程中遇到的每个数学公式或理论作为思考练习，是提升思维能力和形成良好思考习惯的有效方式。  第三段摘要：文章倡导广泛
学习的概念，不仅限于学术课程，还包括生活中的各种活动。学习被视为增长、进步并从中获得快乐的过程。课业外的活动同样能带来学习与成长，无论是在个人技能、身体健康、情
感智力还是社交能力的提升。作者列举了运动、爬山、旅行等活动的例子，强调这些经历对拓宽视野、增加知识、增进自我理解以及体验不同文化的价值。文章强调快乐与学习的关系
，鼓励将任何带来快乐的经历视为学习的机会。  第四段摘要：文章探讨了恋爱和交友对于个人成长的积极影响。恋爱不仅增进情感理解，还能体验人际关系中的各种情感与期待，
对个人成长大有裨益。即使没有缘分，通过交友也能学习人际互动、沟通技巧以及人与人之间的复杂情感。对于电机系的学生来说，与同系优秀同学建立友谊同样具有巨大价值，参与
系会举办的活动能显著提升自我，激发潜力，这些活动对个人发展同样重要。  第五段摘要：文章强调除了专业学习之外，参与各种活动对于个人成长和发展的价值。这些活动能够
提供宝贵的学习机会，帮助个体提升团队协作、领导力以及解决问题等非技术性能力。这些被称为“软实力”的能力对个人发展至关重要，能培养合作精神与执行力，为未来职业生涯
奠定基础。文章指出，虽然软实力并非天生具备，而是通过后天的学习和实践逐渐培养起来的，投资于个人

In [22]:
# 你可以修改最终摘要的输出路径。
output_path = f"./final-summary-{suffix}-llm-multi-stage.txt"

# 如果需要将简体中文转换为繁体中文，请将此选项设置为 True；否则，设置为 False。
convert_to_tradition_chinese = False

if convert_to_tradition_chinese == True:
    # 创建一个 OpenCC 实例用于简体到繁体中文转换。
    cc = OpenCC('s2t')
    final_summarization = cc.convert(final_summarization)

# 输出最终摘要
with open(output_path, "w") as fp:
    fp.write(final_summarization)

# 显示结果
print(f"最终摘要已保存到 {output_path}")
print(f"\n===== 以下是最终摘要 ({len(final_summarization)} 字) =====\n")
for text in textwrap.wrap(final_summarization, 64):
    print(text)

最终摘要已保存到 ./final-summary-信号与人生-llm-multi-stage.txt

===== 以下是最终摘要 (984 字) =====

第一段摘要：文章强调实践操作在获取知识中的重要性，主张通过动手操作如完成课程作业、实验项目或编程任务，比单纯听讲或阅读更能有效吸
收知识。实践性学习能加深理解，帮助学生在学期末通过完成最终项目时验证学习成果，即“做出来的学问”。以学习复杂算法为例，大规模程序
设计任务可将理论知识转化为实际技能，实现全面理解和掌握。文章鼓励实践操作，认为这样的学习过程使知识成为个人学问，而非仅停留在分数
层面。  第二段摘要：文章强调参加挑战性课程的价值，指出此类课程虽然短期内可能不易带来成绩提升，但对整体学习过程至关重要，有助于
避免知识过拟合，促进全面学习。作者以自己在信号课程中的教学方式为例，侧重解释数学公式的含义而非直接推导，鼓励学生通过阅读和思考理
解概念，培养批判性思维能力。作者认为，将阅读过程中遇到的每个数学公式或理论作为思考练习，是提升思维能力和形成良好思考习惯的有效方
式。  第三段摘要：文章倡导广泛学习的概念，不仅限于学术课程，还包括生活中的各种活动。学习被视为增长、进步并从中获得快乐的过程。
课业外的活动同样能带来学习与成长，无论是在个人技能、身体健康、情感智力还是社交能力的提升。作者列举了运动、爬山、旅行等活动的例子
，强调这些经历对拓宽视野、增加知识、增进自我理解以及体验不同文化的价值。文章强调快乐与学习的关系，鼓励将任何带来快乐的经历视为学
习的机会。  第四段摘要：文章探讨了恋爱和交友对于个人成长的积极影响。恋爱不仅增进情感理解，还能体验人际关系中的各种情感与期待，
对个人成长大有裨益。即使没有缘分，通过交友也能学习人际互动、沟通技巧以及人与人之间的复杂情感。对于电机系的学生来说，与同系优秀同
学建立友谊同样具有巨大价值，参与系会举办的活动能显著提升自我，激发潜力，这些活动对个人发展同样重要。  第五段摘要：文章强调除了
专业学习之外，参与各种活动对于个人成长和发展的价值。这些活动能够提供宝贵的学习机会，帮助个体提升团队协作、领导力以及解决问题等非
技术性能力。这些被称为“软实力”的能力对个人发展至关重要，能培养合作精神与执行力，为未来职业生涯奠定基础。文章指出，虽然软实力并
非天生具备，而是通

##### 精炼方法（the method of Refinement) - Refine

Refinement 就是把每次的文本和之前的摘要结合起来丢给大模型，类似于迭代：

![Refinement](../Guide/assets/image-20240924092753352.png)

步骤（Pipeline）如下：
- 第1步：从一小部分数据开始，运行prompt生成初始输出。
- 第2步：对后续每个文档，将前一个输出与新文档结合输入。
- 第3步：LLM 根据新文档中的信息精炼输出。
- 第4步：此过程持续迭代，直到处理完所有文档。

对应的核心代码：
```python
        # 第1步
        first_paragraph_summarization = summarization(client=client, summarization_prompt=summarization_prompt, ...)
        # 第2步
        chunk_text = f"""前 {index} 段的摘要: {paragraph_summarizations[-1]}\n第 {index + 1} 段的内容: {chunk}"""
        # 第3步
        paragraph_summarization = summarization(client=client, summarization_prompt=summarization_prompt, ...)
```

直接运行代码，交互式修改，不用关心这里的代码细节。

In [23]:
# ===== 精炼摘要的prompt设置 =====

# 你可以修改摘要prompt和最大生成标记数，但不要修改 <text> 部分
instructions = Markdown("**精炼摘要的prompt设置**")

# 设置最大生成标记数
max_tokens_markdown = Markdown("**max_tokens**<br>设置最大生成标记数")
max_tokens_widget = widgets.IntText(
    value=550,
    description='max_tokens:',
    style={'description_width': 'initial'}
)

# 初始摘要prompt
summarization_prompt_markdown = Markdown("**summarization_prompt**<br>你可以修改初始摘要prompt，但不要修改 `<text>` 部分：")
summarization_prompt_widget = widgets.Textarea(
    value="请在 300 字以内，提供以下文字的简洁摘要:<text>",
    description='summarization_prompt:',
    layout=widgets.Layout(width='500px', height='100px'),
    style={'description_width': 'initial'}
)

# 精炼摘要prompt
summarization_prompt_refinement_markdown = Markdown("**summarization_prompt_refinement**<br>你可以修改精炼摘要prompt，但不要修改 `<text>` 部分：")
summarization_prompt_refinement_widget = widgets.Textarea(
    value="请在 500 字以内，结合原先的摘要和新的内容，提供简洁的摘要:<text>",
    description='summarization_prompt_refinement:',
    layout=widgets.Layout(width='500px', height='100px'),
    style={'description_width': 'initial'}
)

# 创建输出区域来显示打印内容
output_area = widgets.Output()

# 获取用户输入值并显示
def on_button_click(b):
    global max_tokens, summarization_prompt_template, summarization_prompt_refinement_template
    with output_area:
        output_area.clear_output()  # 清除之前的输出
        max_tokens = max_tokens_widget.value
        summarization_prompt_template = summarization_prompt_widget.value
        summarization_prompt_refinement_template = summarization_prompt_refinement_widget.value
        
        # 打印配置
        print(f"最大标记数: {max_tokens}")
        print(f"初始摘要prompt: {summarization_prompt_template}")
        print(f"精炼摘要prompt: {summarization_prompt_refinement_template}")

# 创建提交按钮
submit_button = widgets.Button(description="提交配置")
submit_button.on_click(on_button_click)

# 显示 Markdown 和所有小部件
display(instructions)
display(max_tokens_markdown, max_tokens_widget)
display(summarization_prompt_markdown, summarization_prompt_widget)
display(summarization_prompt_refinement_markdown, summarization_prompt_refinement_widget)

# 显示提交按钮
display(submit_button)

# 显示输出区域
display(output_area)

**精炼摘要的prompt设置**

**max_tokens**

IntText(value=550, description='max_tokens:', style=DescriptionStyle(description_width='initial'))

**summarization_prompt**<br>你可以修改初始摘要prompt，但不要修改 `<text>` 部分：

Textarea(value='请在 300 字以内，提供以下文字的简洁摘要:<text>', description='summarization_prompt:', layout=Layout(height='100…

**summarization_prompt_refinement**<br>你可以修改精炼摘要prompt，但不要修改 `<text>` 部分：

Textarea(value='请在 500 字以内，结合原先的摘要和新的内容，提供简洁的摘要:<text>', description='summarization_prompt_refinement:', layou…

Button(description='提交配置', style=ButtonStyle())

Output()

In [24]:
paragraph_summarizations = []

# 首先，我们分别对拆分后的每个部分进行摘要。
for index, chunk in enumerate(chunks):

    if index == 0:
        # 记录开始时间
        start = time.time()

        # 构建初始摘要prompt
        summarization_prompt = summarization_prompt_template.replace("<text>", chunk)

        # 第1步: 从一小部分数据开始，运行prompt生成初始输出
        first_paragraph_summarization = summarization(client=client, summarization_prompt=summarization_prompt, model_name=model_name, temperature=temperature, top_p=top_p, max_tokens=max_tokens)

        # 记录结果
        paragraph_summarizations.append(first_paragraph_summarization)

        # 计算执行时间并四舍五入到两位小数
        cost_time = round(time.time() - start, 2)

        # 打印摘要及其长度
        print(f"----------------------------第 {index + 1} 段摘要----------------------------\n")
        for text in textwrap.wrap(first_paragraph_summarization, 80):
            print(f"{text}\n")
        print(f"第 {index + 1} 段摘要的长度: {len(first_paragraph_summarization)}")
        print(f"生成第 {index + 1} 段摘要所需的时间: {cost_time} 秒。\n")

    else:
        # 记录开始时间
        start = time.time()

        # 第2步：将前一个输出与新文档结合输入
        chunk_text = f"""前 {index} 段的摘要: {paragraph_summarizations[-1]}\n第 {index + 1} 段的内容: {chunk}"""

        # 构建精炼摘要prompt
        summarization_prompt = summarization_prompt_refinement_template.replace("<text>", chunk_text)

        # 第3步：LLM 被指示根据新文档中的信息精炼输出
        paragraph_summarization = summarization(client=client, summarization_prompt=summarization_prompt, model_name=model_name, temperature=temperature, top_p=top_p, max_tokens=max_tokens)

        # 记录结果
        paragraph_summarizations.append(paragraph_summarization)

        # 计算执行时间并四舍五入到两位小数
        cost_time = round(time.time() - start, 2)

        # 打印结果
        print(f"----------------------------前 {index + 1} 段摘要----------------------------\n")
        for text in textwrap.wrap(paragraph_summarization, 80):
            print(f"{text}\n")
        print(f"前 {index + 1} 段摘要的长度: {len(paragraph_summarization)}")
        print(f"生成前 {index + 1} 段摘要所需的时间: {cost_time} 秒。\n")

    # 第4步：此过程持续迭代，直到处理完所有文档。

----------------------------第 1 段摘要----------------------------

这段文字强调了实践在获取和深化知识中的重要性。它指出，单纯听讲或阅读难以确保知识真正内化为个人学问。通过实际操作、完成项目或解决复杂问题，如实验、课程作业或编程

任务，人们能更有效地将所学知识整合并转化为自己的理解。这种实践过程不仅加深了对知识的理解，还能提高解决问题的能力。因此，作者建议在学习过程中积极参与实践，以促进

知识的实际应用与个人掌握。这种学习方法强调的不仅仅是成绩或表现，而是知识与技能的真正内化。

第 1 段摘要的长度: 205
生成第 1 段摘要所需的时间: 2.98 秒。

----------------------------前 2 段摘要----------------------------

实践在学习过程中的重要性不容忽视，它能帮助个体将知识内化并转化为个人理解，有效提升解决问题的能力。然而，许多人因担心实践带来的挑战而选择逃避，认为它可能导致成绩

不佳或感到过于劳累，从而错过宝贵的学习机会。实际上，实践是深化学习、培养思考能力的关键途径。通过阅读时主动思考数学公式背后的含义，以及理解书籍中论述的概念，个体

能够学会如何读书并培养批判性思维习惯。因此，尽管实践过程可能艰辛且不直接带来具体成绩，但它对于全面学习和能力提升至关重要。在学习过程中，积极参与实践，尤其是那些

看似困难的任务，是实现知识与技能真正内化的有效策略。

前 2 段摘要的长度: 266
生成前 2 段摘要所需的时间: 3.54 秒。

----------------------------前 3 段摘要----------------------------

实践在学习过程中的作用至关重要，它不仅帮助个体深化理解、转化知识为个人见解，还能显著提升解决问题的能力。实践并非总是轻松且立竿见影，许多人因为担心可能遇到的挑战

而选择回避，担忧成绩下滑或过度劳累。然而，实践正是学习的基石，它促使个体深入思考，学会批判性阅读与理解，从而培养出强大的思考能力。即使面对艰难任务，积极参与实践

也是实现知识与技能真正内化的关键策略。  学习不仅仅局限于课本之内，课外活动同样充满学习的机会。无论是体育运动、户外探险还是旅行，都能促进个人成长、

In [25]:
# 你可以修改最终摘要的输出路径。

output_path = f"./final-summary-{suffix}-llm-refinement.txt"

# 如果需要将简体中文转换为繁体中文，请将此选项设置为 True；否则，设置为 False。
convert_to_tradition_chinese = False

if convert_to_tradition_chinese == True:
    # 创建一个 OpenCC 实例用于简体到繁体中文转换。
    cc = OpenCC('s2t')
    paragraph_summarizations[-1] = cc.convert(paragraph_summarizations[-1])

# 输出最终摘要
with open(output_path, "w") as fp:
    fp.write(paragraph_summarizations[-1])

# 显示结果
print(f"最终摘要已保存到 {output_path}")
print(f"\n===== 以下是最终摘要 ({len(paragraph_summarizations[-1])} 字) =====\n")
for text in textwrap.wrap(paragraph_summarizations[-1], 80):
    print(text)

最终摘要已保存到 ./final-summary-信号与人生-llm-refinement.txt

===== 以下是最终摘要 (286 字) =====

摘要：学习过程中的实践对知识理解、转化及问题解决能力培养至关重要，参与多样化的课外活动促进个人成长、技能提升，增强团队合作、领导力等软实力，对电机工程师等专业学
生的生涯发展意义重大。35岁至55岁为个人发展的黄金时期，电机工程专业毕业生的发展差异主要由实力、努力、机遇及自我技能四个关键因素决定。真正的成功在于全面发展的
软实力、持续的努力与适时把握机会。实力指电机工程领域的全面学习与掌握，避免过度专业化，全面发展。努力在个人未来发展中起到关键作用，而自我技能的提升，通过课业外活
动实现，对个人成长同样重要。拥有清晰长程目标的人更容易向上发展，这四大因素共同影响个人成就。


## 总结

通过这个代码文件，你将了解如何：

1. 使用 Whisper 模型从视频音频生成字幕。
2. 将字幕文件中的内容提取并处理，转换为文本格式。
3. 使用 OpenAI API 为文本生成摘要。

你也将：
1. 理解 AI 视频总结助手实际上是如何工作的。
2. 发现 AI 应用实际真的并不难，你完全有能力制作一个属于自己的 AI 应用。
